# CIA World Factbook

We'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db


## Overview of the Data
We'll begin by getting a sense of what the data looks like.

In [2]:
%%sql
SELECT * 
FROM facts 
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and see what they tell us.

## Summary Statistics

In [3]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, 
       MIN(population_growth) AS min_pop_growth, MAX(population_growth) AS max_pop_growth
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics on the previous screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

## Exploring Outliers

In [4]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population)
                    FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It seems like the table contains a row for Antarctica, which explains the population of 0. According to the CIA Factbook, there is no indigenous inhabitants, but both permanent and summer-only staffed research stations. 

In [5]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

## Summary Statistics Revisited

In [6]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!


## Exploring Average Population and Area
Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [7]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

## Finding Densely Populated Countries
To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have:

- Above average values for population.
- Below average values for area.

In [8]:
%%sql
SELECT name, round(CAST(population AS FLOAT)/CAST(area AS FLOAT),2) AS pop_area_ratio
FROM facts
WHERE population > (SELECT AVG(population)
                    FROM facts
                    WHERE name <> 'World')
AND   area < (SELECT AVG(area)
              FROM facts
              WHERE name <> 'World')
ORDER BY pop_area_ratio DESC;

 * sqlite:///factbook.db
Done.


name,pop_area_ratio
Bangladesh,1138.07
"Korea, South",492.53
Philippines,336.66
Japan,335.84
Vietnam,284.86
United Kingdom,263.08
Germany,226.47
Italy,205.27
Uganda,153.92
Thailand,132.48


We can see that Bangladesh is the most densely populated country, followed by South Korea. 